In [2]:
!apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [3]:
import os
os.getcwd()

'/teamspace/studios/this_studio'

In [4]:
import os
path = os.getcwd() +'/FinalProject/wav2vec-kenlm/data/ivritai/'
file = path + 'ivritai.txt'

before downloading and unpacking the KenLM repo.

In [5]:
import platform
import os

if platform.system() == 'Darwin':  # Darwin stands for MacOS
    !curl -L https://kheafield.com/code/kenlm.tar.gz | tar xz
elif platform.system() == 'Linux':
    !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
else:
    print("Unsupported operating system")

--2024-04-30 17:29:59--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               

-                   100%[===================>] 480.36K  1.77MB/s    in 0.3s    

2024-04-30 17:29:59 (1.77 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [6]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

mkdir: cannot create directory ‘kenlm/build’: File exists
build_binary  fragment	       lmplz			     query
count_ngrams  interpolate      phrase_table_vocab	     streaming_example
filter	      kenlm_benchmark  probing_hash_table_benchmark


In [7]:
!pip install pyctcdecode
!pip install kenlm -U


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [8]:
import os

In [9]:
!kenlm/build/bin/lmplz -o 5 <"{file}" > "{path}/5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/ivritai/ivritai.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100


****************************************************************************************************
Unigram tokens 666793 types 68410
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:820920 2:1270531840 3:2382247168 4:3811595520 5:5558577152
Statistics:
1 68410 D1=0.676854 D2=1.07847 D3+=1.38516
2 386701 D1=0.846211 D2=1.16431 D3+=1.33312
3 576613 D1=0.94022 D2=1.29629 D3+=1.40398
4 593907 D1=0.980639 D2=1.48502 D3+=1.6486
5 557180 D1=0.971796 D2=1.8707 D3+=1.29316
Memory estimate for binary LM:
type    MB
probing 46 assuming -p 1.5
probing 56 assuming -r models -p 1.5
trie    23 without quantization
trie    13 assuming -q 8 -b 8 quantization 
trie    20 assuming -a 22 array pointer compression
trie    10 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:820920 2:6187216 3:11532260 4:14253768 5:15601040
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [10]:
!head -20 {path}5gram.arpa

\data\
ngram 1=68410
ngram 2=386701
ngram 3=576613
ngram 4=593907
ngram 5=557180

\1-grams:
-5.625689	<unk>	0
0	<s>	-0.64655524
-1.3858544	</s>	0
-2.838002	אה	-0.27534404
-3.1600022	נגיד	-0.23054226
-2.3611543	אתה	-0.6647856
-3.524679	יודע	-0.21384771
-1.9815823	זה	-0.49287868
-2.8288188	כזה	-0.30056417
-4.772435	ימכרו	-0.0725213
-1.943969	את	-0.45797426
-3.1616285	ה	-0.4458315


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [11]:
with open(f"{path}5gram.arpa", "r") as read_file, open(f"{path}5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [12]:
!head -20 {path}5gram_correct.arpa

\data\
ngram 1=68411
ngram 2=386701
ngram 3=576613
ngram 4=593907
ngram 5=557180

\1-grams:
-5.625689	<unk>	0
0	<s>	-0.64655524
0	</s>	-0.64655524
-1.3858544	</s>	0
-2.838002	אה	-0.27534404
-3.1600022	נגיד	-0.23054226
-2.3611543	אתה	-0.6647856
-3.524679	יודע	-0.21384771
-1.9815823	זה	-0.49287868
-2.8288188	כזה	-0.30056417
-4.772435	ימכרו	-0.0725213
-1.943969	את	-0.45797426


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## Create the Processor

In [39]:
# from transformers import AutoProcessor
from transformers import AutoProcessor, Wav2Vec2BertProcessor
# processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew",
#                                             unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
# )
processor=Wav2Vec2BertProcessor.from_pretrained("/teamspace/studios/this_studio/models/facebook/w2v-bert-2.0-finetuned", 
                                            unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
)


In [40]:
processor.tokenizer

Wav2Vec2CTCTokenizer(name_or_path='/teamspace/studios/this_studio/models/facebook/w2v-bert-2.0-finetuned', vocab_size=34, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	30: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	31: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	32: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	33: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
}

Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [41]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder.

In [42]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path=f"{path}5gram_correct.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/ivritai/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [43]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [44]:
# !sudo apt-get install git-lfs tree

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class.

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [45]:
# from huggingface_hub import Repository

# repo = Repository(local_dir="FinalProject/models/ken-lm", clone_from="imvladikon/wav2vec2-xls-r-300m-hebrew")

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [46]:
# processor_with_lm.save_pretrained("xls-r-300m-sv")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [47]:
# !tree -h xls-r-300m-sv/

As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [35]:
# !kenlm/build/bin/build_binary xls-r-300m-sv/language_model/5gram_correct.arpa xls-r-300m-sv/language_model/5gram.bin

Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [36]:
# !rm xls-r-300m-sv/language_model/5gram_correct.arpa && tree -h xls-r-300m-sv/

Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [37]:
# repo.push_to_hub(commit_message="Upload lm-boosted decoder")

In [49]:
#save the processor to local
# processor_with_lm.save_pretrained("/teamspace/studios/this_studio/models/KenLM-Wav2Vec2-imvladikon-300m-ivritai")
processor_with_lm.save_pretrained("/teamspace/studios/this_studio/models/wav2vec2bert-ivritai")